In [44]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
from torch.autograd import Variable
import skvideo.io
import time
import math

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

batch_size = 1  # Extract Resnet features in a tensor of batch_size videos.
total_ext = 1000  #  Extract Resnet features of total_ext videos

resnet_feat_path = '../dataset_224/activitynet/res_feature.pt'
batch_ckpt_path = '../dataset_224/activitynet/batch_num.txt'
video_path = '../dataset_224/activitynet/videos/'

# Download pre trained Resnet101.
resnet50 = models.resnet50(pretrained=True)
modules = list(resnet50.children())[:-1]
resnet50 = nn.Sequential(*modules)
for p in resnet50.parameters():
    p.requires_grad = False
resnet50 = resnet50.to(device)

In [45]:
# Convert id's list to dictionary with key as id, value as id numpy array
def video_array(batch_ids):
    batch_array = {}
    for ids in batch_ids:
        video = skvideo.io.vread(video_path + ids)
        batch_array.update({ids:video})
    return batch_array

In [46]:
# Convert array dictionary to resnet_dictionary with key as id, value as id tensor feature array
def resnet_features(batch_arrayd):
    with torch.no_grad():
      
        batch_feature = {}
        ids = list(batch_arrayd.keys())
        video_array = [x for x in batch_arrayd.values()]
        array_sizes = [x.shape[0] for x in batch_arrayd.values()]

        video1_array = np.array(video_array[0], dtype = np.float32)  # change datatype of frames to float32
        video_tensor = torch.from_numpy(video1_array)

        video_frames = video_tensor.size()[0]
        num_steps = math.ceil(video_frames / 100)
        resnet_feature = torch.zeros(video_frames,2048).to(device)
        
        video_tensor = video_tensor.permute(0,3,1,2) # change dimension to [?,3,224,224]
        
        for i in range(num_steps):
            start = i*100
            end = min((i+1)*100, video_frames)
            tensor_var = Variable(video_tensor[start:end]).to(device)
            feature = resnet50(tensor_var).data
            feature.squeeze_(3)
            feature.squeeze_(2)
            resnet_feature[start:end] = feature    

        return {ids[0]:resnet_feature}
        

In [47]:
# seconds to minutes
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return m, s

# time in minutes b/w since to now
def timeSince(since):
    now = time.time()
    s = now - since
    min, sec = asMinutes(s) 
    return min, sec


if __name__ == '__main__':

    featured = {}
    
    all_ids = os.listdir(video_path)
    total_ext = len(all_ids) 
    
    print('Total videos', total_ext)
    print('batch size', batch_size)
    total_iter = math.ceil(total_ext / batch_size)

    if os.path.exists(batch_ckpt_path):
        with open(batch_ckpt_path,'r') as f:
            content = f.readline().rstrip()
            batch_start = int(content)
    else:
        batch_start =  0

    batch_end = min(batch_start+batch_size,total_ext) 

    iter = int((batch_start/batch_size) + 1)
    while batch_start != batch_end:

        print('Iteration', iter, 'left', total_iter-iter, end=' ')
        start_time = time.time()
        batch_id = all_ids[batch_start:batch_start+batch_size]
        batch_arrayd = video_array(batch_id)
        torch.cuda.empty_cache()
        batch_featuresd = resnet_features(batch_arrayd) 

        featured.update(batch_featuresd)
        print('time taken (%dm %ds)'% timeSince(start_time))

        # Save resnet features to pytorch file
        state = featured
        torch.save(state, resnet_feat_path)  

        # save batch_start to batch_ckpt_path file
        with open(batch_ckpt_path, 'w') as f:
            f.write(str(batch_end))

        batch_start = batch_end 
        batch_end = min(batch_start+batch_size, total_ext)
        iter += 1

Total videos 13879
batch size 1
Iteration 1 left 13878 time taken (0m 6s)
Iteration 2 left 13877 time taken (0m 8s)
Iteration 3 left 13876 time taken (0m 6s)
Iteration 4 left 13875 time taken (0m 6s)
Iteration 5 left 13874 time taken (0m 7s)
Iteration 6 left 13873 time taken (0m 13s)
Iteration 7 left 13872 time taken (0m 5s)
Iteration 8 left 13871 time taken (0m 5s)
Iteration 9 left 13870 time taken (0m 4s)
Iteration 10 left 13869 time taken (0m 3s)
Iteration 11 left 13868 time taken (0m 5s)
Iteration 12 left 13867 time taken (0m 4s)
Iteration 13 left 13866 time taken (0m 16s)
Iteration 14 left 13865 time taken (0m 11s)
Iteration 15 left 13864 time taken (0m 7s)
Iteration 16 left 13863 time taken (0m 9s)
Iteration 17 left 13862 time taken (0m 6s)
Iteration 18 left 13861 time taken (0m 1s)
Iteration 19 left 13860 time taken (0m 8s)
Iteration 20 left 13859 time taken (0m 10s)
Iteration 21 left 13858 time taken (0m 6s)
Iteration 22 left 13857 time taken (0m 15s)
Iteration 23 left 13856 ti

Iteration 187 left 13692 time taken (0m 14s)
Iteration 188 left 13691 time taken (0m 18s)
Iteration 189 left 13690 time taken (0m 13s)
Iteration 190 left 13689 time taken (0m 11s)
Iteration 191 left 13688 time taken (0m 10s)
Iteration 192 left 13687 time taken (0m 10s)
Iteration 193 left 13686 time taken (0m 6s)
Iteration 194 left 13685 time taken (0m 14s)
Iteration 195 left 13684 time taken (0m 9s)
Iteration 196 left 13683 time taken (0m 17s)
Iteration 197 left 13682 time taken (0m 15s)
Iteration 198 left 13681 time taken (0m 11s)
Iteration 199 left 13680 time taken (0m 11s)
Iteration 200 left 13679 time taken (0m 8s)
Iteration 201 left 13678 time taken (0m 16s)
Iteration 202 left 13677 time taken (0m 17s)
Iteration 203 left 13676 time taken (0m 8s)
Iteration 204 left 13675 time taken (0m 11s)
Iteration 205 left 13674 time taken (0m 10s)
Iteration 206 left 13673 time taken (0m 16s)
Iteration 207 left 13672 time taken (0m 1s)
Iteration 208 left 13671 time taken (0m 18s)
Iteration 209 l

RuntimeError: CUDA error: out of memory